<a href="https://colab.research.google.com/github/helmana/DLeeg/blob/master/train__open_fist_test_open_feet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
train data : open fist
test data : open feet

subject : 50
channel : 64

layers: 3 conv2D(3,3) & maxpooling(2,2) - 1 Dense(512) 

epoch: 30
batch size: 20


test acc: 
test loss: 

'''


In [0]:


subject_number= 50
task_number= 3

# load dataset in array
list_raw_fnames_train = [[0]*task_number]*subject_number
list_raw_fnames_test = [[0]*task_number]*subject_number

for x in range(subject_number):
    list_raw_fnames_train[x] = mne.datasets.eegbci.load_data(x+1,[3,7,11]) #train task: open fist
    list_raw_fnames_test[x] = mne.datasets.eegbci.load_data(x+1,[5,9,13]) #test task: open feet

list_rawdata_train = np.zeros((subject_number,task_number), dtype='object')
list_rawdata_test = np.zeros((subject_number,task_number), dtype='object')


for i in range(subject_number):
    for j in range(task_number):
        list_rawdata_train[i][j] = mne.io.read_raw_edf(list_raw_fnames_train[i][j], preload=True)
        list_rawdata_test[i][j] = mne.io.read_raw_edf(list_raw_fnames_test[i][j], preload=True)




In [0]:
import math


task_time_train = 60 # train task time
task_time_test = 60 # test task time

sampel_number_per_sec =  160 # sampel rate

total_sampel_number_train =  sampel_number_per_sec *task_time_train # train task time *160
total_sampel_number_test =  sampel_number_per_sec *task_time_test # test task time *160

sample_shift = 4 #step len
window_len= 20

# img number for each one
train_number = math.floor((total_sampel_number_train - sampel_number_per_sec) / sample_shift) +1 - window_len +1
test_number = math.floor((total_sampel_number_test - sampel_number_per_sec) / sample_shift) +1 - window_len +1
 



In [0]:

ch_number = 64

train_img = np.zeros((train_number*subject_number*task_number, ch_number, window_len + 1, 160), dtype = float)
train_label =[]


test_img = np.zeros((test_number*subject_number*task_number, ch_number, window_len + 1, 160), dtype = float)
test_label =[]


train_img.shape


In [0]:
def normalize_channel_data(ch , i, ch_min, ch_max):
  ch = ((ch - ch_min[i]) / (ch_max[i] - ch_min[i] ))
  return ch

In [0]:
# train img & train label


for s in range(subject_number):
  for t in range(task_number):
    rawdataChannels_t_train, times_t_train =list_rawdata_train[s][t][:64,:9600]
    ch_max =[]
    ch_min =[]
    for p in range(len(rawdataChannels_t_train)):
        ch_max = np.append(ch_max, max(rawdataChannels_t_train[p])) # max for each cannel
        ch_min = np.append(ch_min, min(rawdataChannels_t_train[p])) # min for each cannel

    for j in range (train_number):
        for i in range(ch_number):
            for z in range (window_len):
                rawdataChannels_train, times_train =list_rawdata_train[s][t][i,0+(j+z)*sample_shift:sampel_number_per_sec+(j+z)*sample_shift]

                # normalize method
                rawdataChannels_train = normalize_channel_data(rawdataChannels_train, i, ch_min, ch_max)
                train_img[s*task_number*train_number + t*train_number + j][i][z] = rawdataChannels_train
                if z==window_len-1 :
                  train_img[s*task_number*train_number + t*train_number + j][i][z+1] = s
                      



In [0]:
#shaffle train img array
train_img_shuffle = train_img

np.random.shuffle(train_img_shuffle)

In [0]:

train_img_2 = np.zeros((train_number*subject_number*task_number, ch_number, window_len, 160), dtype = float)

#train img label 

train_img_shuffle_len =len(train_img_shuffle)
for i in range(train_img_shuffle_len):
    train_label = np.append(train_label, (train_img_shuffle[i][0][window_len][0] ))
    for j in range (ch_number):
      train_img_2[i][j] = np.delete(train_img_shuffle[i][j], window_len, axis=0)



train_img_2.shape

In [0]:
# test img & test label


for s in range(subject_number):
  for t in range(task_number):
    rawdataChannels_t_test, times_t_test =list_rawdata_test[s][t][:64,:9600]
    ch_max =[]
    ch_min =[]
    for p in range(len(rawdataChannels_t_test)):
        ch_max = np.append(ch_max, max(rawdataChannels_t_test[p])) # max for each cannel
        ch_min = np.append(ch_min, min(rawdataChannels_t_test[p])) # min for each cannel

    for j in range (test_number):
        for i in range(ch_number):
            for z in range (window_len):
                rawdataChannels_test, times_test =list_rawdata_test[s][t][i,0+(j+z)*sample_shift:0+sampel_number_per_sec+(j+z)*sample_shift]

                # normalize method
                rawdataChannels_test = normalize_channel_data(rawdataChannels_test, i, ch_min, ch_max)
                test_img[s*task_number*test_number + t*test_number + j][i][z] = rawdataChannels_test
                if z==window_len-1 :
                  test_img[s*task_number*test_number + t*test_number + j][i][z+1] = s
                      
                      



In [0]:
#shaffle test img array
test_img_shuffle = test_img

np.random.shuffle(test_img_shuffle)

In [0]:

test_img_2 = np.zeros((test_number*subject_number*task_number, ch_number, window_len, 160), dtype = float)

# img label 

test_img_shuffle_len =len(test_img_shuffle)
for i in range(test_img_shuffle_len):
    test_label = np.append(test_label, (test_img_shuffle[i][0][window_len][0] ))
    for j in range (ch_number):
      test_img_2[i][j] = np.delete(test_img_shuffle[i][j], window_len, axis=0)

test_img_2.shape

In [0]:
from keras.utils import to_categorical


In [0]:
train_label = to_categorical(train_label, subject_number)
test_label = to_categorical(test_label, subject_number)

In [0]:


validation_index = math.floor( len(train_img_2) * 0.3)


# valid & train
x_train =train_img_2[validation_index:]
y_train =train_label[validation_index:]

x_valid =train_img_2[:validation_index]
y_valid =train_label[:validation_index]

x_valid.shape




In [0]:
test_index =  len(test_image)//5


# valid & train
test_image_partial =test_image[:test_index]
test_label_partial =test_label[:test_index]

test_image_partial.shape

In [0]:
from keras import layers
from keras import models
from keras import regularizers


model = models.Sequential()
model.add(layers.Conv2D(64, (3,3), activation = 'relu', padding='same', input_shape = (ch_number,window_len,160), data_format= "channels_first" ))
print(model.output.shape)
model.add(layers.MaxPooling2D((2,2)))
print(model.output.shape)
model.add(layers.Conv2D(128, (3,3), activation = 'relu'))
print(model.output.shape)
model.add(layers.MaxPooling2D((2,2)))
print(model.output.shape)
model.add(layers.Conv2D(256, (3,3), activation = 'relu'))
print(model.output.shape)
model.add(layers.MaxPooling2D((2,2)))


model.add(layers.Flatten())
print(model.output.shape)
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dropout(0.5))
print(model.output.shape)
model.add(layers.Dense(subject_number, activation = 'softmax'))
print(model.output.shape)


In [0]:
from keras import optimizers

model.compile(loss= 'categorical_crossentropy',
              optimizer= optimizers.RMSprop(lr=  1e-4),
              metrics = ['acc'])

In [0]:
history = model.fit(
        x_train,
        y_train,
        epochs = 30,
        batch_size = 20,
        validation_data = (x_valid, y_valid)
)

In [0]:
model.evaluate(test_image_partial, test_label_partial)

In [0]:
import matplotlib.pyplot as plt

history_dict = history.history 

loss_values = history_dict ['loss'] 

val_loss_values = history_dict['val_loss']

epochs = range(1, len(loss_values) + 1)



plt.plot(epochs, loss_values, 'bo', label='Training loss') 

plt.plot(epochs, val_loss_values, 'b', label='Validation loss') 

plt.title('Training and validation loss') 

plt.xlabel('Epochs') 

plt.ylabel('Loss') 

plt.legend()

plt.show()

In [0]:
history_dict = history.history 

acc_values = history_dict ['acc'] 

val_acc_values = history_dict['val_acc']



plt.plot(epochs, acc_values, 'bo', label='Training acc') 

plt.plot(epochs, val_acc_values, 'b', label='Validation acc') 

plt.title('Training and validation acc') 

plt.xlabel('Epochs') 

plt.ylabel('acc') 

plt.legend()

plt.show()